In [2]:
import pandas as pd
import numpy as np
import torch
import json
import nltk
import re
import csv
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

import Infersent
from Infersent.models import InferSent

metadata = pd.read_csv("../data/movie.metadata.tsv", sep = '\t', header = None)
metadata.columns = ["movie_id",1,"movie_name",3,4,5,6,7,"genre"]

plots = []
with open("../data/plot_summaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab') 
    for row in tqdm(reader):
        plots.append(row)

movie_id = []
plot = []

# extract movie Ids and plot summaries
for i in tqdm(plots):
    movie_id.append(i[0])
    plot.append(i[1])

# create dataframe
movies = pd.DataFrame({'movie_id': movie_id, 'plot': plot})

# change datatype of 'movie_id'
metadata['movie_id'] = metadata['movie_id'].astype(str)

# merge meta with movies
movies = pd.merge(movies, metadata[['movie_id', 'movie_name', 'genre']], on = 'movie_id')

genres = [] 

# extract genres
for i in movies['genre']: 
    genres.append(list(json.loads(i).values())) 

# add to 'movies' dataframe  
movies['genre_new'] = genres

movies_new = movies[~(movies['genre_new'].str.len() == 0)]

def clean_text(text):
    # remove a string like {{plot}}
    text = re.sub("\s*{{\w*}}\s*", "", text)
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text)
    return text

movies_new['clean_plot'] = movies_new['plot'].apply(lambda x: clean_text(x))

42303it [00:01, 39716.62it/s]
100%|██████████| 42303/42303 [00:00<00:00, 1242014.04it/s]
/anaconda3/envs/workspace/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
new_vocabulary = []

for movie_id in movies_new['movie_id']:
    
    plot_sr = movies_new[movies_new['movie_id']==movie_id]['clean_plot']
    
    for str_obj in plot_sr:
        plot=str_obj
    
    sentence_list = sent_tokenize(plot)

    new_vocabulary = new_vocabulary + sentence_list

In [7]:
V = 2
MODEL_PATH = '/Users/soumava/MovieSummaries/InferSent/encoder/infersent%s.pkl' % V
params_model = {'bsize': 128, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/Users/soumava/MovieSummaries/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

model.build_vocab_k_words(K=100000)
model.update_vocab(new_vocabulary, tokenize=True)

Vocab size : 100000
Found 83944(/149475) words with w2v vectors
New vocab size : 183947 (added 83944 words)


In [17]:
embedding_dict = {}

count = 0

for movie_id in movies_new['movie_id']:
    
    plot_sr = movies_new[movies_new['movie_id']==movie_id]['clean_plot']
    
    for str_obj in plot_sr:
        plot=str_obj
    
    sentence_list = sent_tokenize(plot)

    embedding_array = model.encode(sentence_list, tokenize=True)

    embedding_dict[movie_id] = embedding_array

    count += 1

    print(count)

    if count%10000==0:
        print('PLOTS processed : {}/{}'.format(count, movies_new.shape[0]))

AssertionError: 

In [16]:
np.save('xyz.npy', embedding_dict)